In [24]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
import dask.dataframe as dd
from sklearn.preprocessing import MinMaxScaler

idées de coco:
- on pourrait supprimer les yeasts qui ont les même mutations exactement (mais je sais pas si ça existe)
- si on supprime les features corrélées, c'est bien pour prédire "y" mais pour comprendre à quoi "y" est dû, c'est pas top non? Ex: si deux mutations sont corrélées (très liées), pour la compréhension de "on arrive à ce phénotype y si...", ce sera "si on a les deux mutations", mais si on en tej une, on aura "on arrive à ce phénotype y si : la yeast exprime la mutation (celle qu'on a pas tej)"

In [6]:
'''
# Charger les datasets
cnv_data = pd.read_csv("copy_number_variation_dataset.csv")
proteome_data = pd.read_csv("proteome_dataset.csv")
phenotype_data = pd.read_csv("phenotype_dataset.csv")

# y prends la forme : yeast ID, doubling_time


# Joindre les datasets : Copy Number Variations et Proteome
# Fusion basée sur le `Standard_name` ou une autre clé partagée
merged_data = cnv_data.set_index("Standard_name").join(proteome_data.set_index("Protein_ID"), how="inner")
merged_data = merged_data.reset_index()

# Ajouter la cible (YPD_doublingtime) à partir du fichier de phénotypes
merged_data = merged_data.merge(
    phenotype_data[["Standard_name", "YPD_doublingtime"]], 
    left_on="index", 
    right_on="Standard_name", 
    how="inner"
)

# Supprimer les colonnes inutiles
merged_data = merged_data.drop(columns=["Standard_name", "index"])

# Prétraitement des données
# 1. CNV - Normalisation
cnv_features = merged_data.iloc[:, :-2]  # Toutes les colonnes sauf la séquence et YPD_doublingtime
scaler = StandardScaler()
cnv_normalized = scaler.fit_transform(cnv_features)

# 2. Proteome - TF-IDF pour encoder les séquences protéiques
vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(3, 3))  # Tri-grammes pour motifs locaux
proteome_encoded = vectorizer.fit_transform(merged_data["Sequence"])

# Réduction de dimension avec PCA (pour limiter la taille des données après TF-IDF)
pca = PCA(n_components=50)
proteome_reduced = pca.fit_transform(proteome_encoded.toarray())

# Combiner CNV normalisé et protéome réduit
final_data = pd.DataFrame(cnv_normalized).join(pd.DataFrame(proteome_reduced))

# Ajouter la cible (YPD_doublingtime)
#final_data["YPD_doublingtime"] = merged_data["YPD_doublingtime"]

# Séparation en ensembles d'entraînement et de test
X = final_data.drop(columns=["YPD_doublingtime"])
y = final_data["YPD_doublingtime"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
'''


#MODIFS PAO

# y prends la forme : yeast ID, doubling_time
# Charger les datasets
x_train = pd.read_csv("data/X_matrix.csv")
y_train = pd.read_csv("data/Y_matrix.csv")
print("csv read complete")

#check if the first columns is the same
is_same = x_train.iloc[:, 0].equals(y_train.iloc[:, 0])
print(f"Are the first columns the same? {is_same}")



csv read complete
Are the first columns the same? True


In [26]:

def scale_last_columns(data, num_last_columns=7000):
    """
    Scales the last N columns (assumed to be CNVs) to a range of 0 to 1.
    
    Parameters:
        data (pd.DataFrame): x_train data
        num_last_columns (int): Number of copy number variation columns from the end to scale .

    Returns:
        pd.DataFrame: Data with the last columns scaled.
    """
    # Select the last N columns
    cnv_columns = data.iloc[:, -num_last_columns:]
    
    # Scale these columns
    scaler = MinMaxScaler()
    scaled_cnv = scaler.fit_transform(cnv_columns)
    
    # Replace the last N columns with their scaled values
    data.iloc[:, -num_last_columns:] = scaled_cnv
    
    return data


def remove_low_variance_features_last_columns(data, num_last_columns=7000, threshold=0.05):
    """
    Removes features with variance below a specified threshold in the last N columns.
    This has been done already for mutations during our extraction of data, so it is only useful to do it for the copy number variation columns.
    
    Parameters:
        data (pd.DataFrame): Input data.
        num_last_columns (int): Number of columns from the end to apply variance filtering.
        threshold (float): Minimum variance a feature must have to be retained.

    Returns:
        pd.DataFrame: Data with low-variance features removed in the last N columns.
    """
    # Select the last N columns
    target_columns = data.iloc[:, -num_last_columns:]
    
    # Apply VarianceThreshold to these columns
    selector = VarianceThreshold(threshold=threshold)
    reduced_data = selector.fit_transform(target_columns)
    
    # Get the selected column indices
    selected_columns = target_columns.columns[selector.get_support()]
    
    # Replace the last N columns with the reduced set
    data = data.drop(columns=target_columns.columns)  # Drop the original last N columns
    data = pd.concat([data, pd.DataFrame(reduced_data, columns=selected_columns)], axis=1)
    
    return data


def apply_pca_last_columns(data, num_last_columns=7000, n_components=0.95, normalize=True):
    """
    Applies PCA to reduce dimensionality of the last N columns of the dataset.
    
    Parameters:
        data (pd.DataFrame): Input data.
        num_last_columns (int): Number of columns from the end to apply PCA.
        n_components (float or int): Number of components to keep or the amount of variance to retain.

    Returns:
        pd.DataFrame: Data with PCA applied to the last N columns.
    """
    # Separate the last N columns and the rest of the dataset
    other_columns = data.iloc[:, :-num_last_columns]
    target_columns = data.iloc[:, -num_last_columns:]
    
    # Apply PCA to the last N columns
    pca = PCA(n_components=n_components)
    reduced_data = pca.fit_transform(target_columns)

    # Normalize the PCA-transformed features if specified
    if normalize:
        scaler = MinMaxScaler()
        reduced_data = scaler.fit_transform(reduced_data)
    
    # Create a new DataFrame for the reduced PCA data
    reduced_columns = [f'PCA_{i+1}' for i in range(reduced_data.shape[1])]
    reduced_df = pd.DataFrame(reduced_data, columns=reduced_columns, index=data.index)
    
    # Concatenate the other columns with the reduced PCA columns
    result = pd.concat([other_columns, reduced_df], axis=1)
    
    return result



def preprocessed_data (x_df, y_df) :
    x_train = x_df
    x_train = scale_last_columns(x_train)
    x_train = remove_low_variance_features_last_columns(x_train)
    x_train = apply_pca_last_columns(x_train)

    y_train= y_df

    return x_train, y_train

x_train_preprocessed, _ = preprocessed_data (x_train, y_train)

#compute the shape before and after preprocessing
print("Dimensions of the DataFrame:", x_train.shape)
print("Dimensions of the DataFrame after preprocessing :", x_train_preprocessed.shape)

#compute means before and after prepocessing
x_cut=x_train.iloc[1:, 1:]
mean_value_x_train = x_cut.values.mean()
print("Mean of all values:", mean_value_x_train)

x_cut_p=x_train_preprocessed.iloc[1:, 1:]
mean_value_x_train_preprocessed = x_cut_p.values.mean()
print("Mean of all values after preprocessing :", mean_value_x_train_preprocessed)

max_value = np.max(x_cut.values)
min_value = np.min(x_cut.values)

print("Max value:", max_value, "Min value:", min_value)
print("Min value:", min_value)

max_value = np.max(x_cut_p.values)
min_value = np.min(x_cut_p.values)

print("Max value after pp:", max_value, "Min value:", min_value)

Dimensions of the DataFrame: (792, 348523)
Dimensions of the DataFrame after preprocessing : (792, 334900)
Mean of all values: 0.03342165073829667
Mean of all values after preprocessing : 0.02587988254763545
Max value: 1.0 Min value: 0.0
Min value: 0.0
Max value after pp: 1.0000000000000002 Min value: 0.0


In [17]:
#preprocess coco
from scipy.sparse import csr_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import TruncatedSVD
import dask.dataframe as dd

X = pd.read_csv("data/X_matrix.csv")
y = pd.read_csv("data/ordered_phenotype_dataset.csv")

# 1. Convert the mutation matrix to sparse representation
print("Converting to sparse matrix...")
sparse_mutation_matrix = csr_matrix(X.iloc[:, 1:].values)

# 2. Dimensionality Reduction
# a. Low-Variance Filter
print("Applying low-variance filter...")
variance_threshold = 0.01
selector = VarianceThreshold(threshold=variance_threshold)
reduced_sparse_matrix = selector.fit_transform(sparse_mutation_matrix)
selected_features = selector.get_support(indices=True)

# b. Truncated SVD (for further reduction)
print("Applying PCA using Truncated SVD...")
n_components = 9  # Adjust based on the desired dimensionality
svd = TruncatedSVD(n_components=n_components, random_state=42)
reduced_matrix_pca = svd.fit_transform(reduced_sparse_matrix)

# 3. Convert Reduced Matrix Back to DataFrame
print("Converting reduced matrix back to DataFrame...")
reduced_df = pd.DataFrame(
    reduced_matrix_pca, 
    columns=[f'PC_{i}' for i in range(reduced_matrix_pca.shape[1])]
)
reduced_df.insert(0, 'Yeast_ID', X['Yeast_ID'])

# 4. Normalize Data
print("Normalizing data...")
reduced_df.iloc[:, 1:] = (reduced_df.iloc[:, 1:] - reduced_df.iloc[:, 1:].mean()) / reduced_df.iloc[:, 1:].std()

# 5. Save Intermediate Results
print("Saving to file...")
reduced_df.to_parquet('reduced_mutation_matrix.parquet')

# 6. Use Dask for Further Processing (if needed)
print("Processing in batches using Dask...")
dask_df = dd.read_parquet('reduced_mutation_matrix.parquet')

# Handle missing data: Fill NaN with 0 (or another value)
processed_dask_df = dask_df.fillna(0)  # Adjust fill value as needed

# Compute and Save Final Dataset
final_df = processed_dask_df.compute()
final_df.to_parquet('final_mutation_matrix.parquet')

print("Preprocessing complete.")

X_train, X_test, y_train, y_test = train_test_split(final_df.iloc[:, 1:] , y.iloc[:,1], test_size=0.2, random_state=42)


Converting to sparse matrix...
Applying low-variance filter...
Applying PCA using Truncated SVD...
Converting reduced matrix back to DataFrame...
Normalizing data...
Saving to file...
Processing in batches using Dask...
Preprocessing complete.


In [18]:
# Modèle : Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Évaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Importance des caractéristiques pour les variations causatives
feature_importances = model.feature_importances_
sorted_indices = feature_importances.argsort()[::-1]
print("Top 10 Feature Importances:")
for idx in sorted_indices[:10]:
    print(f"Feature {idx}: {feature_importances[idx]}")

Mean Squared Error: 0.005929892617593614
Top 10 Feature Importances:
Feature 2: 0.5384615384615384
Feature 1: 0.46153846153846156
Feature 0: 0.0


phenotype avec juste les YPD doubling time, yeast ID et tri par ordre alphabetique : je mets kla valeur mpyenne quand on a aps d'infos


In [8]:
# Charger le fichier, remplir les valeurs manquantes et trier les Yeast_ID
file_path = "data/filtered_phenotype_dataset.csv"
data = pd.read_csv(file_path)

data["YPD_doublingtime"] = data["YPD_doublingtime"].fillna(data["YPD_doublingtime"].mean())
data = data.sort_values(by="Yeast_ID").reset_index(drop=True)

# Sauvegarder le fichier mis à jour
data.to_csv(file_path, index=False)
print(f"Fichier trié et avec imputation des valeurs manquantes enregistré sous : {file_path}")

Fichier trié et avec imputation des valeurs manquantes enregistré sous : data/filtered_phenotype_dataset.csv
